In [1]:
import sys
sys.path.append("/s/ls4/users/grartem/RL_robots/CommandClassifier")
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import torch
torch.cuda.is_available()
import yaml
import pyhocon
import json
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
import simpletransformers
from sklearn.metrics import classification_report
from simpletransformers.classification import (
    MultiLabelClassificationModel, MultiLabelClassificationArgs
)
from RobotCommandClassifier import utils

In [2]:
configFileContent = pyhocon.ConfigFactory.parse_file("../Configs/SimpleLM.conf")
CONFIG = configFileContent['rubert_tiny2_fold4'].as_plain_ordered_dict()

In [3]:
train_x_df, train_y_df, valid_x_df, valid_y_df, test_x_df, test_y_df = utils.load_data(**CONFIG["Data"])

enc = OneHotEncoder()
enc.fit(train_y_df.values)

labels = []
encoded_labels = enc.transform(train_y_df.values).toarray().astype(int)
for i in range(train_y_df.shape[0]):
    labels.append(encoded_labels[i].tolist())
train_df = pd.DataFrame(list(zip(train_x_df, labels)))
train_df.columns = ["text", "labels"]

In [4]:
model_args = MultiLabelClassificationArgs(**CONFIG["Model"]["Args"])

# Create a MultiLabelClassificationModel
model = MultiLabelClassificationModel(
    CONFIG["Model"]["model_type"],
    #CONFIG["Model"]["model_name"],    
    CONFIG["output_dir"] + '/models/checkpoint-63780-epoch-10',
    num_labels=len(labels[0]),
    use_cuda=True,
    args=model_args,
    
)

In [5]:
predictions, raw_outputs = model.predict(test_x_df.values.tolist())

predictions_2 = np.zeros((len(predictions), len(enc.categories_)))
for i in range(len(predictions)):
    shift = 0
    for j in range(len(enc.categories_)):
        predictions_2[i,j] = np.argmax(raw_outputs[i, shift:shift+len(enc.categories_[j])])
        shift += len(enc.categories_[j])


  0%|          | 0/263 [00:00<?, ?it/s]

  0%|          | 0/33 [00:00<?, ?it/s]

In [6]:
if not os.path.exists(os.path.join(CONFIG["output_dir"], "reports")):
    os.mkdir(os.path.join(CONFIG["output_dir"], "reports"))
result = utils.calculate_metrics_2(test_y_df, predictions_2, display=True)
with open(os.path.join(CONFIG["output_dir"], "reports/epoch-10_classes_report.json"), "w") as f:
    json.dump(result, f)

result_avg = utils.calculate_metrics(test_y_df, predictions_2, config={
    "report_metrics": CONFIG["Report"]["report_metrics"]
})
with open(os.path.join(CONFIG["output_dir"], "reports/epoch-10_avg_report.json"), "w") as f:
    json.dump(result_avg, f)
for k, v in result_avg.items():
    print(np.round(v*100), "\t", k)

y
              precision    recall  f1-score   support

         0.0       0.75      0.60      0.67        10
         1.0       0.89      0.89      0.89        18
         5.0       0.78      0.70      0.74        10
         6.0       0.00      0.00      0.00         0
         7.0       0.80      0.92      0.86        26
         8.0       0.99      0.97      0.98       199
        11.0       0.00      0.00      0.00         0

    accuracy                           0.94       263
   macro avg       0.60      0.58      0.59       263
weighted avg       0.95      0.94      0.94       263

94.0 	 correct_samples_perc
94.0 	 [y]_acc
59.0 	 [y]_macrof1
59.0 	 avg_macro_f1
94.0 	 avg_acc


/s/ls4/users/grartem/anaconda3/envs/simptr/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/s/ls4/users/grartem/anaconda3/envs/simptr/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/s/ls4/users/grartem/anaconda3/envs/simptr/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(resu